In [44]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [45]:
import pickle
import pandas as pd
import numpy as np

In [46]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/harsha/.local/share/virtualenvs/batch-hTUDUmda/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/harsha/.local/share/virtualenvs/batch-hTUDUmda/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [54]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [55]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')


In [56]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [57]:
np.std(y_pred)

5.28140357655334

In [58]:
def output_file_path(taxi_type,year,month):
    return f'gs://mlops-data-2023/taxi_type={taxi_type}/year={year:04d}/month={month:02d}/homework.parquet'

In [59]:
def save_results(df, y_pred,taxi_type,year,month):
    df_result = pd.DataFrame()
    df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df_result['predicted_duration'] = y_pred
    df_result.to_parquet(output_file_path(taxi_type,year,month),engine='pyarrow',compression=None, index=False)
    

In [60]:
save_results(df, y_pred,'yellow',2022,2)